In [212]:
import snscrape.modules.twitter as sntwitter
import cohere
import json
import re
import emoji

URL_PATTERN = r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+'
API_KEY = '{api_Key}'

In [189]:
username = 'elonmusk'
limit = 100


query = f'(from:{username}) until:2020-01-01'
tweets = []
counter = 0
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    counter += 1
    tweets.append({
        'username': tweet.user.username,
        'content': tweet.content,
    })
    if counter == limit:
        break

In [190]:
tweets[0]['username']

'elonmusk'

In [191]:
full_text = ''

for tweet in tweets:
    full_text += tweet['content']

In [192]:
summarizer = '''

Hashtags:'''

In [193]:
cohere_prompt = full_text[:2030] + summarizer

In [194]:
summarizer

'\n\nHashtags:'

In [195]:
with open(f'{username}_twitter_data.json', 'w') as f:
    json.dump(tweets, f)

### Real part

In [223]:
def get_info_from_web(username, limit=100):
    query = f'(from:{username}) until:2020-01-01'
    tweets = []
    counter = 0
    
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        counter += 1
        tweets.append({
            'username': tweet.user.username,
            'content': tweet.content,
        })
        if counter == limit:
            break
    return tweets

In [224]:
def create_cohere_prompt(tweets):
    full_text = ''

    for tweet in tweets:
        full_text += tweet['content']
    
    return full_text

In [196]:
def cohere_prediction(prompt):
    co = cohere.Client(API_KEY)
    response = co.generate(
      model='large',
      prompt=prompt,
      max_tokens=50,
      temperature=0.8,
      k=0,
      p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop_sequences=["--"],
      return_likelihoods='NONE')
    
    return response.generations[0].text

In [197]:
def get_hashtags(prompt):
    summarizer = '''

Hashtags:'''
    prompt = prompt[:2030]
    prompt += summarizer
    pred = cohere_prediction(prompt)
    return list(map(lambda x: '#' + x, re.findall(r"#(\w+)", pred)))

In [198]:
def get_summary(prompt):
    summarizer = '''

In summary:'''
    prompt = prompt[:2030]
    prompt += summarizer
    pred = cohere_prediction(prompt)
    return pred

In [199]:
def get_urls(prompt):
    summarizer = '''

URLs:'''
    prompt = prompt[:2030]
    prompt += summarizer
    pred = cohere_prediction(prompt)
    return re.findall(URL_PATTERN, pred)

In [234]:
def get_mood(prompt):
    summarizer = '''

My mood:'''
    prompt = prompt[:2030]
    prompt += summarizer
    pred = cohere_prediction(prompt)
    return pred

In [221]:
def get_emojis(prompt):
    summarizer = '''

Emojis:'''
    prompt = prompt[:2030]
    prompt += summarizer
    pred = cohere_prediction(prompt)
    return list(set(re.findall(r'[^\w\s,.]', pred)))

In [242]:
def save_json(data):
    username = data['username']
    with open(f'cohere_hack_data/{username}_data.json', 'w') as f:
        json.dump(data, f)

In [202]:
get_hashtags(full_text)

['#Tesla', '#TeslaUnplugged', '#SpaceX']

In [203]:
print(get_summary(full_text))

 it's less than the 550 kWh of the Model 3, but it's enough to last for 1.5 years of typical US driving, which is how long the battery is expected to last. 300 kWh is the best-case scenario for


In [204]:
print(get_urls(full_text))

['https://twitter.com/search?q=Spacex', 'https://en.wikipedia.org/wiki/SpaceX']


In [235]:
print(get_mood(full_text))

 "I need to get into some more serious physical therapy ASAP"
Helpful inventions?
@skypechicken @PhillyTesla @richard_staniforth @JohnnaCrider1 @TeslaGong @lil


In [253]:
print(get_emojis(full_text))

[':', '/', '!', '✨', '’', '@']


In [258]:
def pipe(username):
    tweets = get_info_from_web(username)
    prompt = create_cohere_prompt(tweets)
    data = {
        'username': username,
        'hashtags': get_hashtags(prompt),
        'summary': get_summary(prompt),
        'urls': get_urls(prompt),
        'emojis': get_emojis(prompt),
        'mood': get_mood(prompt)
    }
    
    save_json(data)
    return data

In [267]:
pipe('trump')

{'username': 'trump',
 'hashtags': ['#TrumpNews',
  '#trade',
  '#BREXIT',
  '#News',
  '#Trump',
  '#Elections',
  '#2020Selection',
  '#CollegeFootball',
  '#TrumpTaxes',
  '#TrumpDoral',
  '#TrumpTaxes',
  '#TrumpTaxes'],
 'summary': ' I need to buy a new pair of pants because I’m going to spew out my guts within 30 minutes of stepping on a plane and undergoing intense in-flight turbulence.\n\nThis, however, is life.\n\nMoney.',
 'urls': ['https://t.co/m9iItIDUUphttps',
  '//t.co/Jcs81sZi1Mhttps',
  '//t.co/oy4LQiZOVvhttps',
  '//t.co/Mw'],
 'emojis': ['🐶', '🦁', ':', '🕛', '/', '🐾', '✨', '@'],
 'mood': ' https://t.co/fsOyfOubyJMonday #MotivationMonday https://t.co/Np5xyTjdq8\n\nOpened my new Trump Tower home office yesterday and this view is breathtaking ���'}